In [1]:
import so3g
import numpy as np
import g3_utils as ut
import matplotlib.pyplot as plt
from spt3g import core
from spt3g.core import G3Units as gu
import pathlib
from astropy.coordinates import SkyCoord

In [9]:
control_computer_g3_dir = pathlib.Path("/media/player1/blast2020fc1/blasttng_g3")
roach1_pass3_file = control_computer_g3_dir / "testing/roach1_pass3.g3"

stage_file = lambda i: str(control_computer_g3_dir / f"mapmaking/pipeline/stage{i}.g3")

In [7]:
kid_rejects = [11, 21, 57, 81, 111, 127, 147, 148, 149, 150, 151, 152, 154, 158, 170, 181, 182, 192, 195,
              211, 223, 225, 245, 263, 282, 286, 293, 297, 319, 327, 331, 333, 334, 336, 337, 340, 341, 349, 352]
exclude_kids = [ut.kid_string(reject_id, roach_id=1) for reject_id in kid_rejects]

In [10]:
stats = ut.DetectorStats(data_key="df")

pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=str(roach1_pass3_file))
# create non-normalized df & save
pipe.Add(ut.add_cal_lamp_df, iq_key="cal_lamp_data", exclude_kids=exclude_kids)
pipe.Add(ut.AddScanDF, exclude_kids=exclude_kids)
pipe.Add(core.G3Writer, filename=stage_file(1))
# compute medians & standard deviation for each detector scan
pipe.Add(stats)
pipe.Run()

detector_medians = np.median(np.array(stats.medians), axis=0)
detector_stds = np.median(np.array(stats.stds), axis=0)

In [13]:
# TODO: move towards object-oriented approach, maybe using spt3g.maps FlatSkyMap?
map_args = {
    'ra0': 231.15 * core.G3Units.deg,
    'dec0': -56.2 * core.G3Units.deg,
    'xlen': 1.4 * core.G3Units.deg,
    'ylen': 0.9 * core.G3Units.deg,
    'res': 1 * core.G3Units.arcmin,
}
binner0 = ut.MapBinner(timestreams="norm_df", stds=detector_stds, **map_args)

pipe = core.G3Pipeline()
pipe.Add(core.G3Reader, filename=stage_file(1))
pipe.Add(ut.add_radec_so3g)
pipe.Add(ut.NormalizeDF, detector_medians=detector_medians)
pipe.Add(binner0)
pipe.Run()

AssertionError: must set source_coords!